## Pull All Youtube Videos from a Playlist
#### @author: christiangrech

In [241]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser
import json
import urllib
import isodate
import datetime
import pandas as pd

keys_file = open("key.txt")
lines = keys_file.readlines()


DEVELOPER_KEY = lines[0].rstrip()
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

In [256]:
PLAYLIST_ID = "" # 
PLAYLIST_ID = "" # 
PLAYLIST_ID = "" # 
PLAYLIST_ID = "" # 
PLAYLIST_ID = "PLl8Jd_mlhkWdoenTKAKMpjEH3ugpZo2q2" # 
#PLAYLIST_ID = "PLchAhemgM-fm_3Mpf9_yha0D9gu_EL6-j" # illum ma steph
#PLAYLIST_ID = "PL-ee8SSNXhJhL1_67L2CfHi3MlvPuQ7pR" # personaggi
#PLAYLIST_ID = "PLedzF-qwWGve9f8CRZuY2bat-tqlbBuDF" # hospice
#PLAYLIST_ID = "PLwpLuTweIvtl7dLfuL7m6xUlZ7K3rkF4-" # water agency

#PLAYLIST_ID = "PLqKghUN-biUsz4MhMve2sWkHgaQjwKJHg" # ymca
#PLAYLIST_ID = "PLqKghUN-biUv7ZRGMDiQDKy5KWCbn43El" # ymca

#PLAYLIST_ID = "PLqKghUN-biUvhfNTmxjxWdzPLv6i6O79p" # ymca

#PLAYLIST_ID = "PLzrpE0iRHx_od4bsbPx7GHZUMZINtlTlB" # live events
#PLAYLIST_ID = "PLbibS98gdzTjsxKLfy3eSi-VCsBUTqBAv" # Xtra
#PLAYLIST_ID = "PLbibS98gdzTgSF1VR4Aobf0DM_QZZfzjD" # Xtra 2021
#PLAYLIST_ID = "PLuZB25kuU2GHNaqVzO_31IbYU20WcZQzG" # edward vlogs
#PLAYLIST_ID = "PLzrpE0iRHx_rzznd0m_IBYjB0cYDV3BWa" # malti pur
#PLAYLIST_ID = "PLC4PPe7hpY6aJdMD3UDcYJz52vFzlcLXB" #educ 

#PLAYLIST_ID = "PL0N1rjXcbqbq9kYl0MdaVaUBP0q1SBOdt" # santa monika
#PLAYLIST_ID = "PL0N1rjXcbqboOpLh-Pae8BbnodE5mifxp" # comedy
#PLAYLIST_ID = "PL5YTElVBLOyhRdPqTgPwvGj0qmoJcYjiN" # revved
#PLAYLIST_ID = "PL5vr18QrM_e6OR8XQHU_6s_mKSI84Y7Y9" # MEA education
#PLAYLIST_ID = "PLBul04hJcpcI8LtFYiBxkTqDVeEb3Ygbl" # politics
#PLAYLIST_ID = "PLI0_rlU15IebpNgVOzd0BM2pWjPsHS-tS" # education
#PLAYLIST_ID = "PLI0_rlU15IeYUWUnVs44kUy88a01wLfi_" # education
#PLAYLIST_ID = "PLI0_rlU15Ieaubt9skhNK9uFvNZsrcnej" # politics
#PLAYLIST_ID = "PLI0_rlU15IebO9PbHCIU0UX7-67INqEav" # politics
#PLAYLIST_ID = "PLI0_rlU15IeZIOHkY9VXji-WDZUCL8CyF" # marittimu

#PLAYLIST_ID = "PLl9rwUAyPo1UnNRDJ_GiiYtl6sPWjMYhR" # Whats for dinner, skip first
#ACELINE

#PLAYLIST_ID = "PLl9rwUAyPo1V7rzNdlLwMU_beVSAQZu76" # mil palazz



In [257]:
print(PLAYLIST_ID)

PLl8Jd_mlhkWdoenTKAKMpjEH3ugpZo2q2


In [258]:
def fetch_all_youtube_videos(playlistId):
    """
    Fetches a playlist of videos from youtube
    We splice the results together in no particular order

    Parameters:
        parm1 - (string) playlistId
    Returns:
        playListItem Dict
    """
    youtube = build(YOUTUBE_API_SERVICE_NAME,
                    YOUTUBE_API_VERSION,
                    developerKey=DEVELOPER_KEY)
    res = youtube.playlistItems().list(
    part="contentDetails",
    playlistId=playlistId,
    maxResults="50"
    ).execute()

    nextPageToken = res.get('nextPageToken')
    while ('nextPageToken' in res):
        nextPage = youtube.playlistItems().list(
        part="contentDetails",
        playlistId=playlistId,
        maxResults="50",
        pageToken=nextPageToken
        ).execute()
        res['items'] = res['items'] + nextPage['items']

        if 'nextPageToken' not in nextPage:
            res.pop('nextPageToken', None)
        else:
            nextPageToken = nextPage['nextPageToken']

    return res

In [259]:
if __name__ == '__main__':
    
  # comedy central playlist, has 332 video
  # https://www.youtube.com/watch?v=tJDLdxYKh3k&list=PLD7nPL1U-R5rDpeH95XsK0qwJHLTS3tNT
    info = {}
    info['links'] = []
    info['duration'] = []
    duration = 0
    total_dur = datetime.timedelta(0)
    videos = fetch_all_youtube_videos(PLAYLIST_ID)
    for video in videos['items']:
        video_id = video['contentDetails']['videoId']
        url = f'https://www.youtube.com/watch?v={video_id}'
        search_url = f'https://www.googleapis.com/youtube/v3/videos?id={video_id}&key={DEVELOPER_KEY}&part=contentDetails'
        req = urllib.request.Request(search_url)
        response = urllib.request.urlopen(req).read().decode('utf-8')
        data = json.loads(response)
        all_data = data['items']
        info['links'].append(url)
        dur= all_data[0]['contentDetails']['duration']
        duration = isodate.parse_duration(dur)
        total_dur = total_dur + duration
        info['duration'].append(str(duration))
    


IndexError: list index out of range

In [ ]:
df = pd.DataFrame(data=info)
df

In [254]:
df.to_csv('/Users/christiangrech/Documents/GitHub/Youtube-Playlist-Info-Extractor/dict1.csv', index=False)

In [255]:
print(total_dur)

7:18:52


17:09:06
6:30:45
1 day, 0:35:35
7:05:35
7:53:53
5:12:35
2:17:15
2:59:38
16:12:20
11:40:25

In [260]:
info

{'links': ['https://www.youtube.com/watch?v=wta2Oe4VNUM',
  'https://www.youtube.com/watch?v=ah4bhGzYQjs',
  'https://www.youtube.com/watch?v=-S7Ajh-Xwg8',
  'https://www.youtube.com/watch?v=t9AGLZ5xVS0',
  'https://www.youtube.com/watch?v=OSt3J0XTHG8',
  'https://www.youtube.com/watch?v=XKhGXoGRgqY',
  'https://www.youtube.com/watch?v=rWwa8Nl0rEM',
  'https://www.youtube.com/watch?v=yULCDXdFmWw',
  'https://www.youtube.com/watch?v=rOqNumN2Q98',
  'https://www.youtube.com/watch?v=MUgbm3HrBFA',
  'https://www.youtube.com/watch?v=dpkoyyqEGHU'],
 'duration': ['0:13:02',
  '0:14:52',
  '0:14:51',
  '0:13:03',
  '0:12:38',
  '0:12:03',
  '0:02:38',
  '0:10:29',
  '0:12:05',
  '0:19:06']}

In [203]:
#PLAYLIST_ID = "PLKTZIA5MRAocmtEQep08g44sYJ_58u7j1" # Rallies politika
#PLAYLIST_ID = "PLZGlYv8NpNKQDngcd6w6n1_vpsU5BFBKL" # cartoons
#PLAYLIST_ID = "PL-ee8SSNXhJhpLTBjvugTNl2o99Zqyjxq" # Prime time
#PLAYLIST_ID = "PLzrpE0iRHx_qc6TCuBqW9JkJZH9q-uk8r" # Natura doc
#PLAYLIST_ID = "PL-ee8SSNXhJhn4CZphkPQY0D_rliJ5QHI"  # first four


In [202]:
#PLAYLIST_ID = "PLKTZIA5MRAoc70n1WDdQNJQ85xdTPKZyO" # politika
#PLAYLIST_ID = "PLZGlYv8NpNKT9P3xraGf0_NuWgh92u0pE"  # stejjer
#PLAYLIST_ID = "PLZGlYv8NpNKTwXWCJ3wjsgtd-r0yj8PeK" # rakkonti
#PLAYLIST_ID = "PLZGlYv8NpNKSdB6buG7wWbzptdTU5plzT" # stejjer 4 sena
#PLAYLIST_ID = "PLZGlYv8NpNKRKIzdrHuocVMeZlFSOycP0" # stejjer 3 sena
#PLAYLIST_ID = "PLZGlYv8NpNKSjSdvQxNUeoPrkHcRAfWx5" # stejjer 2 sena
#PLAYLIST_ID = "PLZGlYv8NpNKT9P3xraGf0_NuWgh92u0pE" # stejjer 1 sena

#PLAYLIST_ID = "PLZGlYv8NpNKSRbqwdo3iffmjfIrOE9M6j" # stejjer 0 sena
#PLAYLIST_ID = "PLZGlYv8NpNKQO9OtCRPKO-6Xowzk6P1vU" # stejjer 

#PLAYLIST_ID = "PLqAYx-Of932EVMyd-yvA-19IEJ5xL4Nii" # dokumentarju
#PLAYLIST_ID = "PLqAYx-Of932GQjh7b7y9ztWosXVqRi_GR" # tahdidiet malti
#PLAYLIST_ID = "PLqAYx-Of932GPomSmP_Hlest9kL9IqDFt" # malti education
#PLAYLIST_ID = "PLrUsKbm79xaaQwCpPJiITaKEbsjtvP4TA" # news politics
#PLAYLIST_ID = "PL-LaAFHEgQQ1B0AAlOM1dt442taZpo8zE" #  literature lectures
#PLAYLIST_ID = "PL-LaAFHEgQQ3aDt7S6vyTYiSNMgflOels" # book interviews with authors
#PLAYLIST_ID = "PL-LaAFHEgQQ25exso1wAKgRHJtS_24BYt" # book reviews
#PLAYLIST_ID = "PL-LaAFHEgQQ2MnLa1WJmwvHVgJB7t3OtK" # book reviews 2
#PLAYLIST_ID = "PLnDio5-ZtQYSbNHWkelTDAwJdXq9XiuwC" # comedy
#PLAYLIST_ID = "PLnDio5-ZtQYTwLqWv8w79mpQH_MP3W91d" # entertainment
#PLAYLIST_ID = "PLnDio5-ZtQYQiEq0Rhb0mA722TNYWgYZ7" # comdey
#PLAYLIST_ID = "PLoNiP6uCOXpCZkWmA0AYkUgw23ydp_MwV" # comedy
#PLAYLIST_ID = "PLXPcBlxlXrLmvpo1ycBw6hEIrANafn437" # comedy
#PLAYLIST_ID = "PLcxdoyejXuPS2eEO5-0J-l43dsZUcsqq0" # entertainemnt
